In [43]:
import json
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Model, optimizers, losses, metrics, Sequential
JSON_PATH = "./data_rnn/imdb_dataset_prepared.json"

In [44]:
# Set random seed for reproducibility
np.random.seed(0)
tf.random.set_seed(0)

In [45]:
# Load data from JSON file
print("Loading data...")
with open(JSON_PATH, "r") as f:
    data = json.load(f)
print("Data loaded successfully.")


Loading data...
Data loaded successfully.


In [46]:
X_train = np.array(data["X_train"], dtype=np.int32)
y_train = np.array(data["y_train"], dtype=np.float32)
X_test  = np.array(data["X_test"],  dtype=np.int32)
y_test  = np.array(data["y_test"],  dtype=np.float32)
embeddings = np.array(data["embeddings"], dtype=np.float32)
vocab = np.array(data["vocab"])

X_train -= 1
X_test -= 1

In [47]:
print("After shift → max index in X_train:", np.max(X_train), "min:", np.min(X_train))
print("After shift → max index in X_test: ", np.max(X_test),  "min:", np.min(X_test))

After shift → max index in X_train: 12848 min: 0
After shift → max index in X_test:  12848 min: 0


In [48]:
embedding_dim = embeddings.shape[1]
vocab_size = embeddings.shape[0]
sequence_length = X_train.shape[1]
batch_size = 128

In [49]:
print(f"X_train: {X_train.shape}")
print(f"y_train: {y_train.shape}")
print(f"X_test: {X_test.shape}")
print(f"y_test: {y_test.shape}")
print(f"embeddings: {embeddings.shape}")
print(f"vocab: {vocab.shape}")

X_train: (40000, 130)
y_train: (40000,)
X_test: (10000, 130)
y_test: (10000,)
embeddings: (12849, 50)
vocab: (12849,)


In [50]:
# Build model using Sequential API (simpler approach)
model = Sequential([
    layers.Embedding(
        input_dim=vocab_size,
        output_dim=embedding_dim,
        weights=[embeddings],
        trainable=True
    ),
    layers.SimpleRNN(
        units=16,
        activation="relu",
        return_sequences=False
    ),
    layers.Dense(1, activation="sigmoid")
])

In [51]:
# Force the model (and therefore each layer) to create its weight tensors:
model.build(input_shape=(batch_size, sequence_length))

model.compile(
    optimizer=optimizers.RMSprop(),
    loss=losses.BinaryCrossentropy(),
    metrics=['accuracy']
)

In [52]:
# Training parameters
epochs = 12

print("\nStarting training...")

# Simple training with model.fit()
history = model.fit(
    X_train, y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(X_test, y_test),
    shuffle=True,
    verbose=1  # This will print progress for each epoch
)

print("\nTraining completed!")


Starting training...
Epoch 1/12
313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.5083 - loss: 0.7096 - val_accuracy: 0.5009 - val_loss: 0.6955
Epoch 2/12
313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.5182 - loss: 0.6917 - val_accuracy: 0.5054 - val_loss: 0.6936
Epoch 3/12
313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.5235 - loss: 0.6878 - val_accuracy: 0.5101 - val_loss: 0.6905
Epoch 4/12
313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.5449 - loss: 0.6770 - val_accuracy: 0.6941 - val_loss: 0.6175
Epoch 5/12
313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.6964 - loss: 0.6085 - val_accuracy: 0.7126 - val_loss: 0.5970
Epoch 6/12
313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.7157 - loss: 0.5872 - val_accuracy: 0.7106 - val_loss: 0.7788
Epoch 7/12
313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.7502 - loss: 0.5484 - val_accuracy: 0.7689 - val_loss: 0.5250
Epoch 8/12
313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.7635 - l

In [ ]:
import time 

loss_fn = tf.keras.losses.BinaryCrossentropy()
optimizer = tf.keras.optimizers.RMSprop()

train_loss_metric = tf.keras.metrics.Mean(name="train_loss")
train_acc_metric  = tf.keras.metrics.BinaryAccuracy(name="train_accuracy")
test_loss_metric  = tf.keras.metrics.Mean(name="test_loss")
test_acc_metric   = tf.keras.metrics.BinaryAccuracy(name="test_accuracy")

batch_size = 128
train_dataset = (
    tf.data.Dataset.from_tensor_slices((X_train, y_train))
    .shuffle(buffer_size=10_000, seed=0)
    .batch(batch_size)
)
test_dataset = (
    tf.data.Dataset.from_tensor_slices((X_test, y_test))
    .batch(batch_size)
)

# 5) Training loop
epochs = 12
for epoch in range(1, epochs + 1):
    start_time = time.time()

    # Reset metrics at the start of each epoch
    train_loss_metric.reset_state()
    train_acc_metric.reset_state()
    test_loss_metric.reset_state()
    test_acc_metric.reset_state()

    # --- Training ---
    for batch_x, batch_y in train_dataset:
        with tf.GradientTape() as tape:
            logits = model(batch_x, training=True)
            loss_value = loss_fn(batch_y, logits)
        grads = tape.gradient(loss_value, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        train_loss_metric.update_state(loss_value)
        train_acc_metric.update_state(batch_y, logits)

    # --- Validation ---
    for batch_x, batch_y in test_dataset:
        val_logits = model(batch_x, training=False)
        val_loss = loss_fn(batch_y, val_logits)
        test_loss_metric.update_state(val_loss)
        test_acc_metric.update_state(batch_y, val_logits)

    epoch_time = time.time() - start_time
    train_loss = train_loss_metric.result().numpy()
    train_acc  = train_acc_metric.result().numpy()
    test_loss  = test_loss_metric.result().numpy()
    test_acc   = test_acc_metric.result().numpy()

    print(
        f"Epoch: {epoch} ({epoch_time:.2f}s)\t"
        f"Train: (l: {train_loss:.2f}, a: {train_acc:.2f})\t"
        f"Test:  (l: {test_loss:.2f}, a: {test_acc:.2f})"
    )

Epoch: 1 (33.90s)	Train: (l: 0.70, a: 0.51)	Test:  (l: 0.69, a: 0.50)
Epoch: 2 (33.76s)	Train: (l: 0.69, a: 0.52)	Test:  (l: 0.69, a: 0.50)
Epoch: 3 (32.27s)	Train: (l: 0.69, a: 0.52)	Test:  (l: 0.69, a: 0.51)


2025-06-14 16:15:11.296525: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch: 4 (33.25s)	Train: (l: 0.69, a: 0.53)	Test:  (l: 0.69, a: 0.51)
Epoch: 5 (32.11s)	Train: (l: 0.68, a: 0.54)	Test:  (l: 0.67, a: 0.62)
Epoch: 6 (31.68s)	Train: (l: 0.66, a: 0.59)	Test:  (l: 0.69, a: 0.53)
Epoch: 7 (32.18s)	Train: (l: 0.65, a: 0.62)	Test:  (l: 0.70, a: 0.54)
Epoch: 8 (32.00s)	Train: (l: 0.62, a: 0.66)	Test:  (l: 0.65, a: 0.60)
Epoch: 9 (31.63s)	Train: (l: 0.60, a: 0.69)	Test:  (l: 0.64, a: 0.65)
Epoch: 10 (31.44s)	Train: (l: 0.57, a: 0.72)	Test:  (l: 0.59, a: 0.71)
Epoch: 11 (31.75s)	Train: (l: 0.54, a: 0.75)	Test:  (l: 0.55, a: 0.76)
Epoch: 12 (31.73s)	Train: (l: 0.52, a: 0.77)	Test:  (l: 0.50, a: 0.77)
